# Comparison of Divergence (Conservation) and Skew-Symmetric (Split) Forms of the Inviscid Burgers' Equation

In [55]:
using CLOUD
using OrdinaryDiffEq
using LinearAlgebra
using TimerOutputs
using Plots, LaTeXStrings
path = "../results/burgers_split_wavenumber_sweep_t5/"

"../results/burgers_split_wavenumber_sweep_t5/"

In [56]:
L = 2.0  # domain length
T = 5.0  # end time
M = 20  # number of elements
p = 4  # degree of discretization
a = 1.0
h_star = L/(M*(p+1))
n_s = 100

ec_flux = EntropyConservativeNumericalFlux{BurgersFlux{1}}()
central_flux = LaxFriedrichsNumericalFlux{BurgersFlux{1}}(0.0)
lf_flux = LaxFriedrichsNumericalFlux{BurgersFlux{1}}(1.0)

reference_approximation = ReferenceApproximation(
    DGSEM(p), Line(), N_plot=50)
spatial_discretization = SpatialDiscretization(uniform_periodic_mesh(
    reference_approximation.reference_element, 
    (0.0,L), M), reference_approximation)
strategy = Eager()
form = StrongFluxDiffForm()

dt_scale=0.01
dt = dt_scale*(L/M)/(2*p+1)
n_t = floor(Int, T/dt)

45000

In [57]:
for i in 1:floor(Int,(M*(p+1)/L))
    results_path = string(path,"run_",i,"/")
    k = i*π
    K = k*h_star
    initial_data = InitialDataGassner(1,k)
    conservation_law = burgers_equation(source_term=SourceTermGassner(1,k),
    numerical_flux=lf_flux)
    
    solver = Solver(conservation_law, spatial_discretization, form, strategy)
    results_path = save_project(conservation_law,
         spatial_discretization, initial_data, form, 
         (0.0, T), strategy, results_path, overwrite=true, clear=true)
    
    u0 = initialize(initial_data, conservation_law, spatial_discretization);
    ode_problem = semidiscretize(solver, u0, (0.0, T))
    save_solution(u0, 0.0, results_path, 0)
    sol = solve(ode_problem, Tsit5(), adaptive=false, 
        dt=T/n_t, save_everystep=false,
        callback=save_callback(results_path, floor(Int, n_t/(n_s-1))))
    save_solution(last(sol.u), last(sol.t), results_path, "final");
    
end